In [1]:
%load_ext autoreload
%autoreload 2
import torch
import torchvision
from torchvision.transforms import transforms
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import confusion_matrix
from data_loader import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import model
torch.set_printoptions(linewidth=120)
from utils import *
from collections import OrderedDict
from run_builder import RunBuilder
from run_manager import RunManager

from tqdm import tqdm
import time

from torch.utils.tensorboard import SummaryWriter

In [2]:
#device = 'cuda' if torch.cuda.is_available else 'cpu'
device = 'cpu'

In [3]:
params = OrderedDict(
    lr = [.01, .001]
    ,batch_size = [10, 100]
    ,epochs = [3]
)

m = RunManager()

for run in RunBuilder.get_runs(params):
    comment = f'-{run}'
    
    data_loader = DataLoader('fashion_mnist',batch_size=run.batch_size)
    network = model.Network().to(device)
    optimizer = optim.Adam(
        network.parameters(), lr=run.lr
    )
    
    m.begin_run(run, network, data_loader)
    
    outer = tqdm(total=run.epochs, desc='Epoch', position=0)
    for epoch in range(run.epochs):
        m.begin_epoch()
        for batch in data_loader.train_loader:
            inner = tqdm(total=batch[1].shape[0], desc='Batch', position=1)
            images = batch[0].to(device)
            labels = batch[1].to(device) # Get Batch
            preds = network(images) # Pass Batch
            loss = F.cross_entropy(preds, labels) # Calculate Loss
            optimizer.zero_grad() # Zero Gradients
            loss.backward() # Calculate Gradients
            optimizer.step() # Update Weights

            m.track_loss(loss, batch)
            m.track_num_correct(preds, labels)
            inner.update(1)
        outer.update(1)
        m.end_epoch()
    m.end_run()
m.save('results')
        



Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

AttributeError: 'tqdm' object has no attribute 'Update'